# FIT5196 Task 1 in Assessment 1
#### Student Name: Sai Ram D Gitte
#### Student ID: 31009751

Date: 13/09/2020

Version: 1.0

Environment: Python 3.7.4 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* os (for reading the names of the file) 
* re (for regular expression, included in Anaconda Python 3.7.4) 
* langid (for detecting language of a text) 

In [ ]:
import os
import langid
import re

# Reading the data in


* Using os.listdir to read only the filenames of all the 2,400 files in the directory specified by the user. 

* Use a relative path to access these directories

* Passing this list of filenames while reading to make sure all the mentioned files are read 

In [ ]:
list_dir = os.listdir('C:/Users/Sai Ram/OneDrive/Semester-2/FIT5196/Assignment-1/Assignment-Data/')

filenames = list_dir

In [ ]:
# Reading data from file1
data1 = []
for fname in filenames:
    with open(fname, 'r', encoding='utf-8',errors='ignore') as fp:
        data1.append(fp.readlines( ))

    fp.close()

# Cleaning the read in data 

* Splitting the data on '},{' gives us an entire tweet which consists of the time it was created at, the tweet id and the tweet 
* The cleaning process does not end here

### Then, for each tweet, cleaning the quotes (" ") around several characters need to be done
* For the ' ":" ' colon part
* For the ' " created ' part
* For the ' Z " ' part
* For the ' " text ' part
* For the id: part where the id: ends with a " followed by a digit

The above mentioned is carried out by using regex and replace functions

In [ ]:
lst = list( ) # list of tweets 
x = str(data1).split('},{') # splitting on },{
for i in range(len(x)): 
    z = x[i].split('","')
    lst.append(z)

In [ ]:
for i in range(len(lst)):
    for j in range(3):

        lst[i][j] = lst[i][j].replace('":"', ':').replace('"created', 'created').replace('Z"', 'Z').replace \
            ('"text', 'text')
        lst[i][j] = lst[i][j].replace(','.join(re.findall(r'^id:\d+"', lst[i][j])).replace(',', ''),
                                      ','.join(re.findall(r'^id:\d+', lst[i][j])).replace(',', ''))

## The Data still needs to be cleaned!
* At the begining of each files that is read, there are unwanted characters appended to them. 

Removal of which is essential and is carried out below

In [ ]:
new_lst = list( )
for i in lst:
    for j in range(len(i)):
        if re.findall(r"{\"data\":", i[j]):
            i[j] = i[j].replace(','.join(re.findall(r"'\[\[\\'{\"data\":\[", i[j])).replace(',', ''), '')
            i[j] = i[j].replace(','.join(re.findall(r"\"\[\[\'\[{", i[j])).replace(',',''),'')

        i[j] = i[j].replace("\n\n", "")
        i[j] = i[j].replace(','.join(re.findall(r'\[.', i[j])).replace(',', ''), '')

        # i[j] = i[j].replace(','.join(re.findall(r'\[{', i[j])).replace(',', ''), '[')

        if re.match(r'id:\d+', i[j]):
            new_lst.append(i)

        if re.findall(r'T\d+:\d+:\d+.\d+Z', i[j]):
            i[j] = i[j].replace(','.join(re.findall(r'T\d+:\d+:\d+.\d+Z', i[j])).replace(',', ''), '')

# Desired content extraction
Instead of cleaning/removing everything that is not necessary, we can filter out what is necessary and store them. 
- since there are data such as "errors", "resource id", "type", "section", "detail" etc in the data along with the tweets, our focus will be to concentrate only on the created_at, id and text part

- within few of the tweets there are id's which have more than 19 digits in them. Only the first 19 digits in them are extracted 

A list is created which would have all the created_at, id, text (one tweet) as a list of lists and sorted in ascending order of the string. 

In [ ]:
clean_lst = list()
for i in new_lst:
    mylist = i
    rt = re.compile("text:.*")
    ri = re.compile("created_at:.*")
    rc = re.compile("id:.*")
    
    newlist_t = list(filter(rt.match, mylist)) 
    newlist_i = list(filter(ri.match, mylist)) 
    newlist_c = list(filter(rc.match, mylist)) 
    
    clean_lst.append(newlist_t + newlist_i + newlist_c)
    
sorted_list_tweets = list(map(lambda y: sorted(y), clean_lst))



In [ ]:
for i in sorted_list_tweets:
    i[1] = i[1].replace(i[1], ','.join(re.findall(r'^id:\d+', i[1])).replace(',', '')) # Replacing looooooong ids with ids
    i[0] = i[0].replace(i[0], ','.join(re.findall(r'^created_at:\d+[-]\d+[-]\d+', i[0])).replace(',', ''))


## There are few tweets with just 2 elements in them
- either id and text
- or text and id
- or date and id and so on

In [ ]:
# Getting rid of all the tweets with 2 elements in them
for i in sorted_list_tweets:
    if len(i) == 2:
        sorted_list_tweets.pop(sorted_list_tweets.index(i))


## Encoding and decoding
- converting all the unicodes to emoji

In [ ]:
r = re.compile(r"\\+u")

for i in sorted_list_tweets:
    
    res = r.sub(r"\\u",i[2])

    y = res.encode("utf-8",'surrogatepass').decode('unicode-escape')
    i[2] = y.encode('utf-16','surrogatepass').decode('utf-16')

# Language classification
- retaining only the tweets that are english (takes about 15min to run)

In [ ]:
dummy = [i for i in sorted_list_tweets if langid.classify(i[2])[0] == 'en']

### Getting the dates

In [ ]:
dict1 = {}

for j in dummy:
    dict1[j[0]] = j[1]+j[2]

dates = []
content = []
for key, value in dict1.items() :
    dates.append(key)
    content.append(value)

# Creating a key value pair
- key: created_at
- value: id and text

for each date collecting all the ids and tweets that were tweeted and each tweet is separated by 3 starts (***): which will help in splitting the string later

In [ ]:
tweets_list = []
updated_date = []
for i in sorted(dates):
    string = ''
    for j in dummy:
        if j[0] == i:
            updated_date.append(i)
            string += j[1]+j[2]+'***'
    tweets_list.append(string)

In [ ]:
# Sorting the date in ascending order
created_date = sorted(list(set(updated_date)))

In [ ]:
# Creating a dictionary with key as the created_date and value as its entire day's tweets 
dictionary_dates_tweets = dict(zip(created_date, tweets_list))

# Writing the entire data to an xml format
- Here the dates, ids and texts are extracted using regex and the written to the file

In [ ]:

with open('31009751.xml','w', encoding="utf-8") as f:
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write('<data>\n')
    for i in created_date:
        i = i.replace(i, re.findall(r'\d{4}-\d{2}-\d{2}',i)[0])
        f.write(f'<tweets date="{i}">\n')

        for l in range(len(dictionary_dates_tweets['created_at:'+i].split('***'))):
            if len(dictionary_dates_tweets['created_at:'+i].split('***')[l].split('text:')) == 2:
                id_1 = re.findall(r'[0-9]{19}',dictionary_dates_tweets['created_at:'+i].split('***')[l].split('text:')[0])[0]
                text_1 = dictionary_dates_tweets['created_at:'+i].split('***')[l].split('text:')[1]


                f.write(f'<tweet id="{id_1}">"{text_1}"</tweet>\n')
            else: pass


